In [ ]:
from xml.sax.saxutils import prepare_input_source

from bs4 import BeautifulSoup
import undetected_chromedriver as uc
from selenium.common import WebDriverException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

In [3]:
driver = uc.Chrome()
wait = WebDriverWait(driver, timeout=10, poll_frequency=0.2)

In [ ]:
url = "https://map.naver.com/p"
driver.get(url)

In [4]:
# search query

def search_query(query):
    query.replace(' ', '%20')
    url = f"https://map.naver.com/p/search/{query}"
    driver.get(url)

search_query("강남 음식점")

In [5]:
# get objects
search_iframe = driver.find_element(By.ID, "searchIframe")
driver.switch_to.frame(search_iframe)

restaurant_list_scroll_container = driver.find_element(By.ID, '_pcmap_list_scroll_container')

In [6]:
# dynamic scroll

scroll_num = 0
prev_scroll_remain = None
while True:
    scroll_top = driver.execute_script("return arguments[0].scrollTop", restaurant_list_scroll_container)
    scroll_height = driver.execute_script("return arguments[0].scrollHeight", restaurant_list_scroll_container)
    scroll_remain = scroll_height - scroll_top
    print(f"scroll_num: {scroll_num}, scroll_height: {scroll_height}, scroll_remain: {scroll_remain}")

    # end-of-loop
    if scroll_remain <= 0 or scroll_num >= 20 or prev_scroll_remain == scroll_remain:
        break

    # do scroll
    for _ in range(10):
        driver.execute_script(f"arguments[0].scrollTop += {scroll_remain // 10}", restaurant_list_scroll_container)
        time.sleep(0.1)

    # delay
    time.sleep(0.4)

    scroll_num += 1
    prev_scroll_remain = int(scroll_remain)

scroll_num: 0, scroll_height: 4279, scroll_remain: 4279
scroll_num: 1, scroll_height: 7522, scroll_remain: 3252
scroll_num: 2, scroll_height: 10286, scroll_remain: 2766
scroll_num: 3, scroll_height: 13083, scroll_remain: 2803
scroll_num: 4, scroll_height: 18394, scroll_remain: 5314
scroll_num: 5, scroll_height: 21422, scroll_remain: 3032
scroll_num: 6, scroll_height: 24386, scroll_remain: 2966
scroll_num: 7, scroll_height: 25133, scroll_remain: 753
scroll_num: 8, scroll_height: 25133, scroll_remain: 401
scroll_num: 9, scroll_height: 25133, scroll_remain: 401


In [24]:
driver.switch_to.default_content()
driver.switch_to.frame(search_iframe)

In [25]:
restaurant_li_list = driver.find_elements(By.CSS_SELECTOR, "#_pcmap_list_scroll_container > ul > li")
restaurant_a_list = driver.find_elements(By.CSS_SELECTOR, "div.CHC5F > div.bSoi3 > a")
prev_restaurant_iframe_src = None

for a in restaurant_a_list[:5]:
    a.click()

    # delay
    driver.switch_to.default_content()
    restaurant_iframe_locator = (By.XPATH, '//*[@id="entryIframe"]')
    try:
        wait.until(
            EC.frame_to_be_available_and_switch_to_it(restaurant_iframe_locator)
        )
    except:
        time.sleep(2)
        # driver.switch_to.default_content()

        driver.switch_to.frame(driver.find_element(*restaurant_iframe_locator))
        print('switched')

    # get info
    # restaurant_info : restaurant_name, restaurant_category, restaurant_address, restaurant_tel
    # review_data : total, key...

    for i in [
        ["restraunt_name", (By.CSS_SELECTOR, "#_title > div > span.GHAhO")],
        ["restraunt_category", (By.CSS_SELECTOR, "#_title > div > span.lnJFt")],
        ["restraunt_address", (By.CSS_SELECTOR, "div > a > span.LDgIH")],
        ["restraunt_tel", (By.CSS_SELECTOR, "div > span.xlx7Q")],
    ]:
        print(i[0], driver.find_element(*i[1]).text)
    print('-----------')


    "#_title > div > span.GHAhO"
    "#_title > div > span.lnJFt"
    "#app-root > div > div > div:nth-child(6) > div > div:nth-child(2) > div.place_section_content > div > div.O8qbU.tQY7D > div > a > span.LDgIH"
    "#app-root > div > div > div:nth-child(6) > div > div:nth-child(2) > div.place_section_content > div > div.O8qbU.nbXkr > div > span.xlx7Q"

    '//*[@id="app-root"]/div/div/div[5]/div/div/div/div/a[5]'

    "#app-root > div > div > div:nth-child(7) > div:nth-child(2) > div.place_section.no_margin.ySHNE > div > div > div.jypaX > em"

    driver.switch_to.default_content()
    driver.switch_to.frame(search_iframe)


restraunt_name 태수
restraunt_category 요리주점
restraunt_address 서울 강남구 도산대로11길 22 1층 태수
restraunt_tel 0507-1317-2149
-----------
restraunt_name 밤별 포차 라운지 강남역본점
restraunt_category 요리주점
restraunt_address 서울 강남구 강남대로102길 13-8 지층, 1층, 2층, 3층
restraunt_tel 0507-1350-0024
-----------
restraunt_name 팔백집 강남점
restraunt_category 육류,고기요리
restraunt_address 서울 강남구 봉은사로2길 15 1층
restraunt_tel 0507-1323-7654
-----------
restraunt_name 쿄코코 신논현점
restraunt_category 일식당
restraunt_address 서울 강남구 강남대로106길 23 지하1층
restraunt_tel 0507-1474-2254
-----------
switched
restraunt_name 핫쵸 스타필드 코엑스몰점
restraunt_category 일식당
restraunt_address 서울 강남구 영동대로 513 코엑스 스타필드 지하 1층
restraunt_tel 0507-1417-6760
-----------


In [ ]:
# driver.switch_to.frame(search_iframe)
soup = BeautifulSoup(driver.page_source, 'html.parser')
# temp_soup = soup.find("#app-root > div > div.XUrfU")
# restaurant_list_soup = soup.find("div#_pcmap_list_scroll_container")
# restaurant_page_soup = soup.find("#app-root > div > div.XUrfU > div.zRM9F")
print(len(soup.find("div", id="_pcmap_list_scroll_container").find("ul").find_all("li")))